In [1]:
# Use seaborn for pairplot
!pip install -q seaborn
import pandas as pd
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style="darkgrid")

https://archive.ics.uci.edu/ml/datasets/car+evaluation

In [2]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'
column_names = ['price', 'maint', 'doors', 'persons', 'lug_capacity', 'safety','output']

raw_dataset = pd.read_csv(url, names=column_names,header=None)

In [3]:
dataset = raw_dataset.copy()
dataset.head(10)

,price,maint,doors,persons,lug_capacity,safety,output
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
5,vhigh,vhigh,2,2,med,high,unacc
6,vhigh,vhigh,2,2,big,low,unacc
7,vhigh,vhigh,2,2,big,med,unacc
8,vhigh,vhigh,2,2,big,high,unacc
9,vhigh,vhigh,2,4,small,low,unacc


Check if there are any empty values in our dataset.

In [4]:
dataset.isna().sum()

price           0
maint           0
doors           0
persons         0
lug_capacity    0
safety          0
output          0
dtype: int64

In [5]:
dataset.describe()

,price,maint,doors,persons,lug_capacity,safety,output
count,1728,1728,1728,1728,1728,1728,1728
unique,4,4,4,3,3,3,4
top,vhigh,vhigh,4,4,small,med,unacc
freq,432,432,432,576,576,576,1210


Transform Categorical arguments into numerical arguments.

In [6]:
price = pd.get_dummies(dataset.price, prefix='price')
maint = pd.get_dummies(dataset.maint, prefix='maint')

doors = pd.get_dummies(dataset.doors, prefix='doors')
persons = pd.get_dummies(dataset.persons, prefix='persons')

lug_capacity = pd.get_dummies(dataset.lug_capacity, prefix='lug_capacity')
safety = pd.get_dummies(dataset.safety, prefix='safety')

labels = pd.get_dummies(dataset.output, prefix='condition')


In [7]:
X = pd.concat([price, maint, doors, persons, lug_capacity, safety] , axis=1)
labels.head(10)

,condition_acc,condition_good,condition_unacc,condition_vgood
0,0,0,1,0
1,0,0,1,0
2,0,0,1,0
3,0,0,1,0
4,0,0,1,0
5,0,0,1,0
6,0,0,1,0
7,0,0,1,0
8,0,0,1,0
9,0,0,1,0


In [8]:
y = labels.values

Split our set into training and testing set.

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [10]:
li = tf.keras.Input(shape=(X.shape[1],))
l1 = tf.keras.layers.Dense(64, activation='relu')(li)
l2 = tf.keras.layers.Dense(32, activation='relu')(l1)
lo = tf.keras.layers.Dense(y.shape[1], activation='softmax')(l2)

In [11]:
model = tf.keras.Model(inputs=li, outputs=lo)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 21)]              0         
_________________________________________________________________
dense (Dense)                (None, 64)                1408      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 132       
Total params: 3,620
Trainable params: 3,620
Non-trainable params: 0
_________________________________________________________________


In [13]:
history = model.fit(X_train, y_train, batch_size=8, epochs=40, verbose=1, validation_split=0.3)

Epoch 1/40
121/121 [==============================] - 1s 6ms/step - loss: 0.8990 - acc: 0.6845 - val_loss: 0.6129 - val_acc: 0.6867
Epoch 2/40
121/121 [==============================] - 0s 3ms/step - loss: 0.4357 - acc: 0.8031 - val_loss: 0.4200 - val_acc: 0.8265
Epoch 3/40
121/121 [==============================] - 0s 3ms/step - loss: 0.2802 - acc: 0.8862 - val_loss: 0.3544 - val_acc: 0.8120
Epoch 4/40
121/121 [==============================] - 0s 3ms/step - loss: 0.2440 - acc: 0.8955 - val_loss: 0.2723 - val_acc: 0.8723
Epoch 5/40
121/121 [==============================] - 0s 3ms/step - loss: 0.1680 - acc: 0.9316 - val_loss: 0.2329 - val_acc: 0.9133
Epoch 6/40
121/121 [==============================] - 0s 3ms/step - loss: 0.1532 - acc: 0.9522 - val_loss: 0.2016 - val_acc: 0.9181
Epoch 7/40
121/121 [==============================] - 0s 3ms/step - loss: 0.1152 - acc: 0.9771 - val_loss: 0.1878 - val_acc: 0.9277
Epoch 8/40
121/121 [==============================] - 0s 3ms/step - loss: 0.

In [14]:
score = model.evaluate(X_test, y_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

11/11 [==============================] - 0s 2ms/step - loss: 0.0546 - acc: 0.9769
Test Score: 0.05459039658308029
Test Accuracy: 0.9768785834312439
